<a href="https://colab.research.google.com/github/danielhsf/peft/blob/main/PEFT_Quicktour.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# PEFT - Quicktour

Esse guia mostra os ponto principais usados para se rodar PEFT usando o google Colab

Link do guia base: https://huggingface.co/docs/peft/quicktour


## Instalando Bibliotecas

In [1]:
!pip install datasets
!pip install evaluate
!pip install sacrebleu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 19.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 11.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 5.1 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; platform_system == 

## Hyparemeters

In [5]:
model_name = "Qwen/Qwen2.5-1.5B-Instruct"

## Carregando Dataset

In [2]:
from datasets import load_dataset

raw_datasets = load_dataset("kde4", lang1="en", lang2="pt")

README.md:   0%|          | 0.00/5.10k [00:00<?, ?B/s]

kde4.py:   0%|          | 0.00/4.25k [00:00<?, ?B/s]

The repository for kde4 contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/kde4.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N] y


Generating train split: 0 examples [00:00, ? examples/s]

In [3]:
split_datasets = raw_datasets["train"].train_test_split(train_size=0.9, seed=20)

In [4]:
split_datasets["validation"] = split_datasets.pop("test")

In [20]:
split_datasets['train']['translation'][0]

{'en': 'Desktop Configuration Editor',
 'pt': 'Editor de Configurações do Ambiente de Trabalho'}

In [7]:
from transformers import AutoTokenizer


tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token

In [21]:
def to_translate_format(example):
    en_sentence = example['translation']["en"]
    pt_sentence = example['translation']["pt"]
    example["translation"] = f"Translate: English to Portuguese:\n{en_sentence}\n### Response:\n{pt_sentence}"
    return example

In [25]:
to_translate_format(split_datasets['train'][0])

{'id': '227298',
 'translation': 'Translate: English to Portuguese:\nDesktop Configuration Editor\n### Response:\nEditor de Configurações do Ambiente de Trabalho'}

In [22]:
split_datasets.map(lambda samples: tokenizer(samples), batched=True)

Map:   0%|          | 0/206476 [00:00<?, ? examples/s]

ValueError: text input must be of type `str` (single example), `List[str]` (batch or single pretokenized example) or `List[List[str]]` (batch of pretokenized examples).

## Treino

Cada método PEFT é definido por uma classe peftConfig que armazena todos os parametros importantes para construir o modelo PeftModel. Por exemplo, para treinar usando LoRA, carregue e crie uma classe LoraConfig e especifique os seguintes parametros:

* task_type: A tarefa para treinamento (*sequence-to-sequence language* modeling nesse caso)
* inference_mode: Se o modelo será usado para inferencia ou não
* r: A dimensão das low-rank matrices
* lora_alpha: Fator de escala para as low-rank matrices
* lora_dropout: Valor de dropout das camadas LoRA


In [ ]:
from peft import LoraConfig, TaskType

peft_config = LoraConfig(task_type=TaskType.CAUSAL_LM,
                         inference_mode=False,
                         r=8,
                         lora_alpha=32,
                         lora_dropout=0.1)

PS: Veja o arquivo de [LoraConfig](https://huggingface.co/docs/peft/v0.15.0/en/package_reference/lora#peft.LoraConfig) para mais detalhes

In [ ]:
from transformers import AutoModelForCausalLM

model = AutoModelForCausalLM.from_pretrained(model_name)

config.json:   0%|          | 0.00/660 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.09G [00:00<?, ?B/s]

Sliding Window Attention is enabled but not implemented for `sdpa`; unexpected results may be encountered.


generation_config.json:   0%|          | 0.00/242 [00:00<?, ?B/s]

In [ ]:
from peft import get_peft_model

model = get_peft_model(model, peft_config)
model.print_trainable_parameters()
"output: trainable params: 2359296 || all params: 1231940608 || trainable%: 0.19151053100118282"

trainable params: 1,089,536 || all params: 1,544,803,840 || trainable%: 0.0705


'output: trainable params: 2359296 || all params: 1231940608 || trainable%: 0.19151053100118282'

In [ ]:
split_datasets["validation"] = split_datasets.pop("test")

In [ ]:
split_datasets["train"][1]["translation"]

{'en': 'The Line Numbers Pane', 'pt': 'A Área de Números de Linha'}

In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(model_name,
                                          padding_side="left",
                                          return_tensors="pt")

tokenizer_config.json:   0%|          | 0.00/7.30k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.78M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.67M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/7.03M [00:00<?, ?B/s]

In [ ]:
en_sentence = split_datasets["train"][1]["translation"]["en"]
pt_sentence = split_datasets["train"][1]["translation"]["pt"]

inputs = tokenizer(en_sentence, text_target=pt_sentence)
inputs

{'input_ids': [785, 7083, 34713, 98542], 'attention_mask': [1, 1, 1, 1], 'labels': [32, 143733, 409, 451, 63918, 409, 8564, 4223]}

In [ ]:
inputs_tgt = tokenizer(pt_sentence)
print(tokenizer.convert_ids_to_tokens(inputs_tgt["input_ids"]))
print(tokenizer.decode(inputs_tgt["input_ids"]))


['A', 'ĠÃģrea', 'Ġde', 'ĠN', 'Ãºmeros', 'Ġde', 'ĠLin', 'ha']
A Área de Números de Linha


In [ ]:
max_length = 256


def preprocess_function(examples):
    prefix = "translate: English to Portuguese:"
    inputs = [prefix+ex["en"] for ex in examples["translation"]]
    targets = [ex["pt"] for ex in examples["translation"]]
    model_inputs = tokenizer(
        inputs, text_target=targets, max_length=max_length, truncation=True
    )
    return model_inputs

In [ ]:
tokenized_datasets = split_datasets.map(
    preprocess_function,
    batched=True,
    remove_columns=split_datasets["train"].column_names,
)

Map:   0%|          | 0/206476 [00:00<?, ? examples/s]

Map:   0%|          | 0/22942 [00:00<?, ? examples/s]

In [ ]:
tokenized_datasets

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 206476
    })
    validation: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 22942
    })
})

In [ ]:
from transformers import DataCollatorForLanguageModeling

tokenizer.pad_token = tokenizer.eos_token
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

In [ ]:
import evaluate

metric = evaluate.load("sacrebleu")

In [ ]:
import numpy as np


def compute_metrics(eval_preds):
    preds, labels = eval_preds
    # In case the model returns more than the prediction logits
    if isinstance(preds, tuple):
        preds = preds[0]

    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)

    # Replace -100s in the labels as we can't decode them
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # Some simple post-processing
    decoded_preds = [pred.strip() for pred in decoded_preds]
    decoded_labels = [[label.strip()] for label in decoded_labels]

    result = metric.compute(predictions=decoded_preds, references=decoded_labels)
    return {"bleu": result["score"]}

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir="danhsf/Qwen2.5-1.5B-en-to-pt",
    learning_rate=1e-5,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    weight_decay=0.01,
    eval_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    save_total_limit=3,
    num_train_epochs=3,
    fp16=True,
    push_to_hub=False,
    report_to=None
)

In [ ]:
from transformers import Trainer


In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    tokenizer=tokenizer,
    data_collator=data_collator,
)

<ipython-input-38-55db22653e3c>:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


In [ ]:
#trainer.train()